In [1]:
import re
import os
import sys
import random
import gpt_2_simple as gpt2
import tensorflow as tf

import numpy as np

In [2]:
from random_word import RandomWords
import requests
import giphy_client

from unsplash.api import Api
from unsplash.auth import Auth

from medium import Client

In [23]:
tags = {#'montag': ['Fake News', 'Opinion', 'Artificial Intelligence', 'NLP', 'Future'], 
        'onezero': ['Artificial Intelligence', 'Technology', 'NLP', 'Future'], 
        #'futura': ['Sci Fi Fantasy', 'Artificial Intelligence', 'NLP', 'Future', 'Storytelling']
       }

In [4]:
def clean(story):
    story = story.replace('&lt;|url|&gt;', 'https://github.com/sirmammingtonham/futureMAG')
    story = story.replace('OneZero', 'FutureMAG')
    story = story.replace('onezero', 'FutureMAG')
    return story[16:]

In [5]:
def split_story(story, run_name):
    story = clean(story)
    split = re.split('(\.)', story)[0:-1]
    metadata = split[0]
    title = metadata[metadata.find('# ')+2:metadata.find('## ')].strip('\n')
    subtitle = metadata[metadata.find('## ')+3:metadata.find('\n', metadata.find('## '))].strip('\n')
    
    split[0] = split[0].replace(subtitle, f"{subtitle} | AI generated article*")
#     if len(title.split(' ')) <= 2:
#         split = story.split('\n', 3)
#         title = split[1]
#         subtitle = split[2]
#         return [title, subtitle, split[3]]
    return [title, f"{subtitle} | AI generated article*", ''.join(split), None, run_name]

In [6]:
def retrieve_images(story):
    #story[1] = subtitle
    #story[2] = story
    matches = [(m.group(), m.start(0)) for m in re.finditer(r"(<\|image\|>)", story[2])]
    image_creds = []
    try:
        client_id = "b9a6edaadf1b5ec49cf05f10aab79d5d2ea1fe66431605d12ec0f7ec22bc7289"
        client_secret = "f00e14688a25656c07f07d85e17b4ebd94e93fcf9bf0fd1859f7713ea1d94c16"
        redirect_uri = "urn:ietf:wg:oauth:2.0:oob"
        auth = Auth(client_id, client_secret, redirect_uri)
        api = Api(auth)
#         q = max(re.sub(r'[^\w\s]', '', story[0]).split(), key=len) #take longest word from subtitle as search term
        q = story[0].split(' ')[:5]
        for match, idx in matches:
            pic = api.photo.random(query=q)[0]
            img = pic.urls.raw
            image_creds.append((f'https://unsplash.com/@{pic.user.username}', pic.user.name))
            cap_idx = story[2].find('*', idx+11)
            story[2] = story[2][:cap_idx] + '&ast;&ast;' + story[2][cap_idx:]
            story[2] = story[2][:idx] + img + story[2][idx+9:]
    except:
        return story
    story[3] = image_creds
    return story

In [7]:
def publish(title, sub, article, creds, run_name):
    if title == sub:
        return
    #holy shit this is excessive
    tag = tags['onezero'] + [max(re.sub(r'[^\w\s]','',title).split(), key=len).capitalize()]
    
    access_token = '2aea40d684c5c501066c6f624d05c952256f0664585d9a36b394c0821ee646499'
    headers = {
        'Authorization': "Bearer " + access_token,
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
    }
    
    base_url = "https://api.medium.com/v1/"

    me_response = requests.request("GET", base_url + 'me', headers=headers).text
    json_me_response = json.loads(me_response)
    user_id = json_me_response['data']['id']
    user_url = base_url + 'users/' + user_id + '/'
    posts_url = user_url + 'posts/'
    pub_url = base_url + 'publications/424c42caa624/posts/'
    
    if not creds:
        return
    else:
        img_creds = ""
        for auth_url, author in creds:
            img_creds += f"[{author}]({auth_url})"
            img_creds += ', ' if len(creds) > 1 else ' '
        img_creds += "on [Unsplash](https://unsplash.com/)"
    
    article += "\n\n*This article was written by a [GPT-2 neural network](https://openai.com/blog/better-language-models). All information in this story is most likely false, and all opinions expressed are fake. Weird to think about…\n\n"
    article += f"**This caption was artificially generated. Image downloaded automatically from {img_creds}.\n\n"
    article += "All links in this article are placeholders generated by the neural network, signifying that an actual link should have been generated there. These placeholders were later replaced by a link to the github project page.\n\n"
    article += "**futureMAG** is an experiment in automated storytelling/journalism. This story was created and published without human intervention.\n\n"
    article += "Code for this project available on github: "
    article += "**[sirmammingtonham/futureMAG](https://github.com/sirmammingtonham/futureMAG)**"
    payload = {
    'title': title,
    'contentFormat': 'markdown',
    'tags': tag if run_name == 'onezero' else tags[run_name],
    'publishStatus': 'draft',
    'content': article
    }
    response = requests.request('POST', pub_url, data=payload, headers=headers)
    print(response.text)
    return payload

In [9]:
run_name = 'onezero_m'
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name)

Instructions for updating:
Colocations handled automatically by placer.
Loading checkpoint checkpoint\onezero_m\model-17000
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint\onezero_m\model-17000


In [10]:
stories = gpt2.generate(
    sess, run_name, return_as_list=True,
    truncate="<|endoftext|>", prefix="<|startoftext|>",
    nsamples=1, batch_size=1, length=8000,
    temperature=1,
    top_p=0.9,
    split_context=0.5,
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
1016
1528


In [19]:
articles = []

for story in stories:
    articles.append(split_story(story, run_name))

for article in articles:
    article = retrieve_images(article)

In [20]:
articles[0][-1] = 'onezero'

In [21]:
articles

[['Hint: The Constellation of Scorpio',
  'The brightest star in the night sky | AI generated article*',
  '# Hint: The Constellation of Scorpio\n\n## The brightest star in the night sky | AI generated article*\n\n![Scalia looks toward Earth through NASA’s Hubble Space Telescope’s viewing window.](https://images.unsplash.com/photo-1496348323715-c11f0fc6aeed?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjgwNzY1fQ)*Scalia looks toward Earth through NASA’s Hubble Space Telescope’s viewing window.&ast;&ast;*\n\nAstronomers love Eta Carinae, a relatively nearby superstar that exploded nearly two centuries ago, and has offered humans a ringside seat to a spectacular cosmic display. The explosion — which scientists call a black hole — has persisted even though the star has since faded from view.\n\nThe explosion of Scorpio, first spotted by NASA’s Voyager mission in 1977, has persisted for years despite being located thousands of light-years away in the constellation Scorpio. New imaging studies by the Hi

In [24]:
publish(*articles[0])

{"data":{"id":"72b08be038c4","title":"Hint: The Constellation of Scorpio","authorId":"113d0ce3c66e5af777e596aa4d9fbbc9bbe91a02c53080b3b5407c7bb8819a607","url":"https://medium.com/@futureBOT/72b08be038c4","canonicalUrl":"","publishStatus":"draft","license":"","licenseUrl":"https://medium.com/policy/9db0094a1e0f","tags":["constellation","nlp","future","technology","artificial-intelligence"],"publicationId":"424c42caa624"}}


{'title': 'Hint: The Constellation of Scorpio',
 'contentFormat': 'markdown',
 'tags': ['Artificial Intelligence',
  'Technology',
  'NLP',
  'Future',
  'Constellation'],
 'publishStatus': 'draft',
 'content': '# Hint: The Constellation of Scorpio\n\n## The brightest star in the night sky | AI generated article*\n\n![Scalia looks toward Earth through NASA’s Hubble Space Telescope’s viewing window.](https://images.unsplash.com/photo-1496348323715-c11f0fc6aeed?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjgwNzY1fQ)*Scalia looks toward Earth through NASA’s Hubble Space Telescope’s viewing window.&ast;&ast;*\n\nAstronomers love Eta Carinae, a relatively nearby superstar that exploded nearly two centuries ago, and has offered humans a ringside seat to a spectacular cosmic display. The explosion — which scientists call a black hole — has persisted even though the star has since faded from view.\n\nThe explosion of Scorpio, first spotted by NASA’s Voyager mission in 1977, has persisted for years despite